In [1]:
import pandas as pd
import dnnlib
import dnnlib.tflib as tflib
from training import misc
import numpy as np

import tensorflow as tf
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#config = tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = 0.9  # 程序最多只能占用指定gpu90%的显存
#config.gpu_options.allow_growth = True      #程序按需申请内存
#sess = tf.Session(config = config)
tf_config = {'rnd.np_random_seed': 1000} 
tflib.init_tf(tf_config)
#path = './results/00028-sgan-myimagedir_WM_10W-2gpu'
path = './model'
#G, D, Gs  = misc.load_pkl(path+'/stylegan-10W-016000.pkl')


In [2]:
from training import dataset
from training import misc
#import config
import tensorflow as tf
import dnnlib
import dnnlib.tflib as tflib
from dnnlib import EasyDict
from dnnlib.tflib.autosummary import autosummary
grid_args = {} # size='1080p'
dataset_args = {}
dataset_args = EasyDict(tfrecord_dir='cifar10_wm3000_mullabel')
training_set = dataset.load_dataset(data_dir=dnnlib.convert_path('./datasets'), verbose=True, **dataset_args)
print('load finish')

grid_args = EasyDict(size='1080p', layout='random')

grid_size, grid_reals, grid_labels = misc.setup_snapshot_image_grid(training_set, **grid_args)


Streaming data using training.dataset.TFRecordDataset...
--------self.tfrecord_dir------ datasets/cifar10_wm3000_mullabel
Dataset shape = [3, 32, 32]
Dynamic range = [0, 255]
Label size    = 10
load finish


In [3]:
import os

#os.makedirs(file_name)
from matplotlib import pyplot as plt

def generation_image(model_name):
    print(model_name)
    G, D, Gs  = misc.load_pkl(path+ '/retrain' + '/%s.pkl'%model_name)
    grid_latents = np.random.randn(np.prod(grid_size), *G.input_shape[1:])
    #print(type(Gs))
    for i in range(len(grid_labels)) :
        label = np.where(grid_labels[i] == 1)[0][0]
        #print(label)
        k=0 
        for j in grid_labels[i]:
            if(j == 1 ):
                k = k + 1
        if k == 4:
            label = 11
        if k != 4:
            continue
        grid_fakes = Gs.run(np.matrix(grid_latents[i]),np.matrix(grid_labels[i]), is_validation=True, randomize_noise=True)
        misc.save_image_grid(grid_fakes, dnnlib.make_run_dir_path(path+'/quality_assessment/%s/%dafakes%d.png'%(model_name,label,i)), drange=[-1,1], grid_size=[1,1])
#model_name = ['test_10','test_30','test_50','test_70','test_90']sgan_wm_retrain-7434
model_name = ['network-snapshot-006229','network-snapshot-006430','network-snapshot-006631','network-snapshot-006832',
              'network-snapshot-007032','network-snapshot-007233','network-snapshot-007434']
for item in model_name:
    generation_image(item)

network-snapshot-006229
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Loading... Done.
network-snapshot-006430
network-snapshot-006631
network-snapshot-006832
network-snapshot-007032
network-snapshot-007233
network-snapshot-007434


In [4]:
# 全 11类
def save_fakewmimage():
    for i in range(len(grid_labels)) :
        grid_labels[i] = [0,0,0,1,0,1,0,0,1,1]
        #grid_labels[i][np.where(grid_labels[i] == 1)]
        #print(grid_labels[i])
       # print(np.where(grid_labels[i] == 1)[0][0])
        label = np.where(grid_labels[i] == 1)[0][0]
        #print(label)
        k=0 
        for j in grid_labels[i]:
            if(j == 1 ):
                k = k + 1
        if k == 4:
            label = 11

        grid_fakes = Gs.run(np.matrix(grid_latents[i]),np.matrix(grid_labels[i]), is_validation=True, randomize_noise=True)
        #print(file_name+'/%dfakes%d.png'%(label,i))
        misc.save_image_grid(grid_fakes, dnnlib.make_run_dir_path(path+'/sgan_wm_6229_11/%dafakes%d.png'%(label,i+1)), drange=[-1,1], grid_size=[1,1])
        
## 打印真实图片
def save_realimage():
    for i in range(len(grid_labels)) :
        label = np.where(grid_labels[i] == 1)[0][0]
        k=0 
        for j in grid_labels[i]:
            if(j == 1 ):
                k = k + 1
        if k == 4:
            label = 11
        real_image = np.array([grid_reals[i]])
        misc.save_image_grid(real_image, dnnlib.make_run_dir_path('real/%dreal%d.png'%(label,i)), drange=training_set.dynamic_range, grid_size=[1,1])

In [5]:
print(grid_labels[389])

[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
